In [12]:
import time
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier

In [2]:
url = 'https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv'
artworks = pd.read_csv(url)
artworks.columns

Index(['Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)',
       'Length (cm)', 'Weight (kg)', 'Width (cm)', 'Seat Height (cm)',
       'Duration (sec.)'],
      dtype='object')

In [3]:
# Select Columns.
artworks = artworks[['Artist', 'Nationality', 'Gender', 'Date', 'Department',
                    'DateAcquired', 'URL', 'ThumbnailURL', 'Height (cm)', 'Width (cm)']]

# Convert URL's to booleans.
artworks['URL'] = artworks['URL'].notnull()
artworks['ThumbnailURL'] = artworks['ThumbnailURL'].notnull()

# Drop films and some other tricky rows.
artworks = artworks[artworks['Department']!='Film']
artworks = artworks[artworks['Department']!='Media and Performance Art']
artworks = artworks[artworks['Department']!='Fluxus Collection']

# Drop missing data.
artworks = artworks.dropna()

In [4]:
artworks.head()

,Artist,Nationality,Gender,Date,Department,DateAcquired,URL,ThumbnailURL,Height (cm),Width (cm)
0,Otto Wagner,(Austrian),(Male),1896,Architecture & Design,1996-04-09,True,True,48.6000,168.9000
1,Christian de Portzamparc,(French),(Male),1987,Architecture & Design,1995-01-17,True,True,40.6401,29.8451
2,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,34.3000,31.8000
3,Bernard Tschumi,(),(Male),1980,Architecture & Design,1995-01-17,True,True,50.8000,50.8000
4,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,38.4000,19.1000


In [5]:
artworks.dtypes

Artist           object
Nationality      object
Gender           object
Date             object
Department       object
DateAcquired     object
URL                bool
ThumbnailURL       bool
Height (cm)     float64
Width (cm)      float64
dtype: object

In [6]:
artworks['DateAcquired'] = pd.to_datetime(artworks.DateAcquired)
artworks['YearAcquired'] = artworks.DateAcquired.dt.year
artworks['YearAcquired'].dtype

dtype('int64')

In [7]:
# Remove multiple nationalities, genders, and artists.
artworks.loc[artworks['Gender'].str.contains('\) \('), 'Gender'] = '\(multiple_persons\)'
artworks.loc[artworks['Nationality'].str.contains('\) \('), 'Nationality'] = '\(multiple_nationalities\)'
artworks.loc[artworks['Artist'].str.contains(','), 'Artist'] = 'Multiple_Artists'

# Convert dates to start date, cutting down number of distinct examples.
artworks['Date'] = pd.Series(artworks.Date.str.extract(
    '([0-9]{4})', expand=False))[:-1]

# Final column drops and NA drop.
X = artworks.drop(['Department', 'DateAcquired', 'Artist', 'Nationality', 'Date'], 1)

# Create dummies separately.
artists = pd.get_dummies(artworks.Artist)
nationalities = pd.get_dummies(artworks.Nationality)
dates = pd.get_dummies(artworks.Date)

# Concat with other variables, but artists slows this wayyyyy down so we'll keep it out for now
X = pd.get_dummies(X, sparse=True)
X = pd.concat([X, nationalities, dates], axis=1)

Y = artworks.Department

In [8]:
type(X)

pandas.core.frame.DataFrame

In [9]:
# Alright! We've done our prep, let's build the model.
# Neural networks are hugely computationally intensive.
# This may take several minutes to run.

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(1000,))
mlp.fit(X, Y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1000,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [10]:
mlp.score(X, Y)

0.6959194288146011

In [11]:
Y.value_counts()/len(Y)

Prints & Illustrated Books    0.521396
Photography                   0.227869
Architecture & Design         0.113225
Drawings                      0.103731
Painting & Sculpture          0.033779
Name: Department, dtype: float64

In [13]:
start_time = time.time()
scores = cross_val_score(mlp, X, Y, cv=3)
elapsed_time = time.time() - start_time

In [14]:
print("runtime {:.6f}".format(elapsed_time))
print(scores)

runtime 254.561445
[0.62139007 0.53861212 0.44820836]


## Drill: Playing with layers

### One

In [15]:
mlp = MLPClassifier(hidden_layer_sizes=(100,))
mlp.fit(X, Y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [16]:
mlp.score(X, Y)

0.6638990023168945

In [18]:
start_time = time.time()
scores = cross_val_score(mlp, X, Y, cv=3)
elapsed_time = time.time() - start_time

In [19]:
print("runtime {:.6f}".format(elapsed_time))
print(scores)

runtime 47.146182
[0.59205674 0.57180549 0.44301643]


### Two

In [26]:
start_time = time.time()
mlp = MLPClassifier(hidden_layer_sizes=(1000,2,))
mlp.fit(X, Y)
elapsed_time = time.time() - start_time

In [27]:
print("runtime {:.6f}".format(elapsed_time))

runtime 98.051788


In [28]:
mlp.score(X, Y)

0.5213958106766278

### Three

In [30]:
start_time = time.time()
mlp = MLPClassifier(hidden_layer_sizes=(1000,4,))
mlp.fit(X, Y)
elapsed_time = time.time() - start_time

In [31]:
print("runtime {:.6f}".format(elapsed_time))

runtime 76.352053


In [32]:
mlp.score(X, Y)

0.5213958106766278

### Four

In [33]:
start_time = time.time()
mlp = MLPClassifier(hidden_layer_sizes=(10000,4,))
mlp.fit(X, Y)
elapsed_time = time.time() - start_time

In [34]:
print("runtime {:.6f}".format(elapsed_time))

runtime 899.435408


In [35]:
mlp.score(X, Y)

0.5213958106766278

### Five

In [36]:
start_time = time.time()
mlp = MLPClassifier(hidden_layer_sizes=(10000,))
mlp.fit(X, Y)
elapsed_time = time.time() - start_time

/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [37]:
print("runtime {:.6f}".format(elapsed_time))

runtime 11612.371814


In [38]:
mlp.score(X, Y)

KeyboardInterrupt: 